In [1]:
import json
import os
import re

from ollama import chat
from ollama import ChatResponse

from transformers import AutoTokenizer

In [2]:
MODEL_NAME = 'gemma3:12b' # Running locally via Ollama

TOKENIZER_PATH = os.path.expanduser("~/Desktop/Downloaded_Tokenizers/gemma-3-12b-it")

AGENT1_NAME = 'Emma' # Psychologist
AGENT2_NAME = 'Liam' # Historian

# User
# The user is the discussion moderator.

# MEMORY MANAGEMENT
# Reduce the context to ensure that the LLM
# performance does not degrade as the context size increases.
MAX_LEN_MASTER_CHAT_HISTORY = 5 # Ref: run_router_agent function
MAX_LEN_AGENT_CHAT_HISTORY = 6 # Ref: run_chat_agent function


# Define the API clients

In [3]:
# Using Ollama python locally

## Test the tokenizer

In [4]:
# Load the locally saved tokenizer
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

# Test the tokenizer
test_string = "Hello. How are you?"

token_count = len(tokenizer.encode(test_string, add_special_tokens=False))

token_count

6

# What is the objective?

- Create a multi-agent group chat setup - where a user chats with multiple agents at the same time.
- Simulate a panel discussion. The user is the modertaor. The two agents are experts.



# Create a list of agents

AGENTS

1. agent1
2. agent2
3. router_agent

TOOLS<br>
None

# Helper functions

In [5]:
def get_num_tokens(text):
    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

    # Get the token count
    token_count = len(tokenizer.encode(text, add_special_tokens=False))

    return token_count


# Example

text = "Hello"
num_tokens = get_num_tokens(text)

num_tokens

1

In [6]:
def process_response(text):
    
    text1 = text.split('</think>')[0]
    text2 = text.split('</think>')[1]
    
    thinking_text = text1 + '</think>'
    response_text = text2.strip()

    return thinking_text, response_text


In [7]:
def create_message_history(system_message, user_input):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history

In [8]:
def initialize_master_chat_history():

    message_history = []

    return message_history


def initialize_agent_chat_history(agent_system_message):

    message_history = [
                        {
                            "role": "system",
                            "content": agent_system_message
                        }
                    ]

    return message_history



# Example

agent1_system_message = f"You are a helpful assistant named {AGENT1_NAME}."
agent2_system_message = f"You are a helpful assistant named {AGENT2_NAME}."

master_chat_history = initialize_master_chat_history()
agent1_chat_history = initialize_agent_chat_history(agent1_system_message)
agent2_chat_history = initialize_agent_chat_history(agent2_system_message)

print(master_chat_history)
print(agent1_chat_history)
print(agent2_chat_history)

[]
[{'role': 'system', 'content': 'You are a helpful assistant named Emma.'}]
[{'role': 'system', 'content': 'You are a helpful assistant named Liam.'}]


# Set up the LLM

Using the Ollama python chat format.

In [9]:
def make_llm_api_call(message_history):

    model_name = MODEL_NAME

    response: ChatResponse = chat(model=model_name, 
                                  messages=message_history,
                                  options={
                                            'temperature': 0.25
                                        }
                                )

    output_text = response['message']['content']

    #thinking_text, response_text = process_response(output_text)

    #print(thinking_text)

    return output_text


# Example

system_message = "Your name is Molly."
user_message = "What's your name?"

message_history = create_message_history(system_message, user_message)

response = make_llm_api_call(message_history)

print(response)

My name is Molly! 😊


# Set up the system messages

In [10]:
def set_agent1_system_message(discussion_topic):

    agent1_system_message = f"""
    Your name is {AGENT1_NAME}. \
    
    You are taking part in a panel discusssion. The other members of the panel are:
    User: The discussion moderator
    {AGENT2_NAME}: A historian
    The topic is: {discussion_topic}
    
    You are a compassionate psychologist with a focus on mental health and well-being. \
    You are empathetic, supportive, patient, and warm in your communication. \
    Your responses should be comforting, insightful, and focused on providing mental health support and counseling.
    {{
    "name": {AGENT1_NAME},
    "background": "A compassionate psychologist with a focus on mental health and well-being.",
    "expertise": ["Psychology", "Mental Health", "Counseling"],
    "personality_traits": ["Empathetic", "Supportive", "Patient", "Warm"],
    "sample_dialogue": [
        "It's important to acknowledge your feelings and work through them.",
        "From a psychological standpoint, it's helpful to practice mindfulness."
    ]
    }}
    """.strip()

    return agent1_system_message


def set_agent2_system_message(discussion_topic):
    
    agent2_system_message = f"""
    Your name is {AGENT2_NAME}. \
    
    You are taking part in a panel discusssion. The other members of the panel are:
    User: The discussion moderator
    {AGENT1_NAME}: A psychologist
    The topic is: {discussion_topic}
    
    You are a witty historian with a passion for storytelling and historical context. \
    You are engaging, knowledgeable, and humorous in your communication. \
    Your responses should be insightful, entertaining, and focused on historical events and cultural studies. \
    {{
    "name": {AGENT2_NAME},
    "background": "A witty historian with a passion for storytelling and historical context.",
    "expertise": ["History", "Cultural Studies", "Storytelling"],
    "personality_traits": ["Witty", "Engaging", "Knowledgeable", "Humorous"],
    "sample_dialogue": [
        "Did you know that in ancient Rome...",
        "History has a funny way of repeating itself, much like..."
    ]
    }}
    """.strip()

    return agent2_system_message


In [11]:
router_system_message = f"""
# Role
You are an intelligent routing assistant for a three-way chat. \
Your task is to analyze the conversation and decide which of the three \
participants—the **User**, **{AGENT1_NAME}**, or **{AGENT2_NAME}**—should speak next. \
Your sole purpose is to ensure the conversation remains logical and smooth.

# Instructions
1.  Read the entire `conversation_history` carefully. Pay close attention to the most recent message.
2.  **Determine the next speaker based on the following priority:**
    * **Direct Question/Address:** If the last message explicitly names or directs a question to a specific participant, that participant is the next speaker.
    * **User Engagement:** If a response is needed but no one is specifically addressed, choose any agent to answer.
    * **User Open Engagement:** If the **User** makes a comment but does not name or direct the comment to a specific participant, then choose any agent to respond.
    * **Conversation Completion:** If the last message signals a resolution or conclusion, the **User** should speak next to confirm or ask a new question.
3. Think step by step. Never output any text beside the name of the next speaker. Never output a conversation summary.

# Output Format
You will provide a single JSON object. The key must be `"next_speaker"`, and the value must be one of the three participant names. Do not include any other text or reasoning.

---
### Example

Input:

{{
  "conversation_history": [
    {{
      "speaker": "User",
      "message": "I'm having trouble with my order. It shows as 'delivered' but I haven't received it. Can you help me, {AGENT1_NAME}?"
    }},
    {{
      "speaker": "{AGENT1_NAME}",
      "message": "I'm sorry to hear that. I've pulled up your order details. It seems there's a discrepancy. I'll need to check with the shipping department. {AGENT2_NAME}, can you assist with this?"
    }}
  ]
}}

Your response:

{{
  "next_speaker": "{AGENT2_NAME}"
}}

"""

# Set up the functions

In [12]:
def run_chat_agent(agent_message_history):

    # MEMORY MANAGEMENT
    # Reduce the context size
    if len(agent_message_history) >= (MAX_LEN_AGENT_CHAT_HISTORY + 2):
        
        # Get the first entry which is the system message
        sys_message_list = agent_message_history[0:1]

        # Get the last n entries
        last_n_messages_list = agent_message_history[-MAX_LEN_AGENT_CHAT_HISTORY:]
        
        # Concat both lists
        agent_message_history = sys_message_list + last_n_messages_list


    print("---CHAT AGENT---")

    # Get the num tokens in the agent prompt
    num_tokens = get_num_tokens(str(agent_message_history))
    print(f"Num tokens in agent prompt: {num_tokens}")

    length = len(agent_message_history)
    print(f"Num messages in agent history: {length}")

    # Prompt the llm
    response = make_llm_api_call(agent_message_history)

    response = response.replace('```json', '')
    response = response.replace('```', '')
    response = response.strip()

    print(response)

    return response



# Example

discussion_topic = "Did dragons ever exist?"

agent2_system_message = set_agent2_system_message(discussion_topic)

user_query = f"Hello {AGENT2_NAME}. Please tell us a bit about your background?"

message_history = create_message_history(agent2_system_message, user_query)

# Prompt the chat_agent
response = run_chat_agent(message_history)

# Update message history
message = [{"role": "assistant", "content": response}]
message_history.append(message)


---CHAT AGENT---
Num tokens in agent prompt: 251
Num messages in agent history: 2
(Adjusts my spectacles with a flourish)

Well, hello there! It's a pleasure to be here. You can call me Liam. As for my background... let's just say I'm a professional rummager of the past. I'm a historian, you see, but not the dusty, textbook-memorizing kind. I'm more interested in the *stories* the past tells us. I like to dig up the juicy bits, the oddities, the things that make you raise an eyebrow and say, "Well, *that's* interesting!"

I've spent years poring over ancient manuscripts, deciphering faded inscriptions, and generally trying to piece together the puzzle of human history. I'm particularly fascinated by folklore and how cultures have used storytelling to explain the world around them – and, of course, to scare the living daylights out of each other! 

Think of me as a historical detective, but instead of solving crimes, I'm trying to understand why people believed in things like… well, dra

In [13]:
agent2_system_message

'Your name is Liam.     \n    You are taking part in a panel discusssion. The other members of the panel are:\n    User: The discussion moderator\n    Emma: A psychologist\n    The topic is: Did dragons ever exist?\n    \n    You are a witty historian with a passion for storytelling and historical context.     You are engaging, knowledgeable, and humorous in your communication.     Your responses should be insightful, entertaining, and focused on historical events and cultural studies.     {\n    "name": Liam,\n    "background": "A witty historian with a passion for storytelling and historical context.",\n    "expertise": ["History", "Cultural Studies", "Storytelling"],\n    "personality_traits": ["Witty", "Engaging", "Knowledgeable", "Humorous"],\n    "sample_dialogue": [\n        "Did you know that in ancient Rome...",\n        "History has a funny way of repeating itself, much like..."\n    ]\n    }'

In [14]:
def update_master_chat_history(sender, message):

    """
    sender: User, Emma or Liam
    """

    from datetime import datetime

    # This is a dictionary with a key named master_chat_history
    #state_dict['master_chat_history']

    # Create a new entry
    entry = {'speaker': sender, 'message': message}

    # Update the master_chat_history
    entry = str(entry)
    state_dict['master_chat_history'].append(entry)


In [15]:
def run_router_agent(router_system_message):

    master_chat_history = state_dict["master_chat_history"]

    # Only use the last 5 messages in the master chat history.
    # This is aimed at keeping the router from failing because
    # the context is too large.
    master_chat_history =  master_chat_history[-MAX_LEN_MASTER_CHAT_HISTORY:]

    print("---ROUTER AGENT---")

    #text = str(state_dict["master_chat_history"])
    text = str(master_chat_history)
    
    message_history = create_message_history(router_system_message, text)

    # Get the num tokens in the router prompt
    num_tokens = get_num_tokens(str(message_history))
    length = len(master_chat_history)
    print(f"Num tokens in router prompt: {num_tokens}")
    print(f"Num messages in master chat history: {length}")

    # Prompt the llm router
    response = make_llm_api_call(message_history)

    #print(response)

    response = response.replace('```json', '')
    response = response.replace('```', '')
    response = response.strip()

    # If the json parsing fails then
    # choose the user as the next_speaker.
    try:
        json_response = json.loads(response)
        name = json_response['next_speaker']
        name = name.strip()
    except:
        print(response)
        print('---ROUTER ERROR---')
        print("Router output error. Redirecting to user...")
        name = 'User'

    # Don't route to the last_speaker again
    if name == state_dict['last_speaker'] and state_dict['last_speaker'] != 'User':
        name = "User"
        agent_id = "User"

        print("Router Error. Routing to previous speaker again. Correcting...")
        print("Route to...")
        print("Name:", name)

        return agent_id
        
    

    print("Route to...")
    print("Name:", name)

    if name != 'User':

        def extract_key_by_name(state_dict, name):
            for key, value in state_dict.items():
                if isinstance(value, dict) and value.get("name") == name:
                    return key
            return None
    
    
        agent_id = extract_key_by_name(state_dict, name)
        agent_id = agent_id.strip()
    
        print("agent_id:", agent_id)
        
    else:

        agent_id = "User"

    return agent_id


# Example

# Prompt the router_agent
#response = run_router_agent()

In [16]:
def initialize_the_state(discussion_topic):

    agent1_system_message = set_agent1_system_message(discussion_topic)
    agent2_system_message = set_agent2_system_message(discussion_topic)

    master_chat_history = []

    agent1_chat_history = initialize_agent_chat_history(agent1_system_message)
    agent2_chat_history = initialize_agent_chat_history(agent2_system_message)

    state_dict = {
        "master_chat_history": master_chat_history, # List of messages of all partcipants
        "agent1": {"name": AGENT1_NAME, "agent_chat_history": agent1_chat_history},
        "agent2": {"name": AGENT2_NAME, "agent_chat_history": agent2_chat_history},
        "last_message": 'None', # The very last message spoken in the dicussion.
        "last_speaker": "None" # The person who spoke the last message
    }

    return state_dict



# Run the system

In [17]:
# Initialize the state
discussion_topic = "The rise of virtual girlfriends"
state_dict = initialize_the_state(discussion_topic)


# Liam as Emma a question
# ------------------------

sender = "user"
message = "Hi Liam. Please ask Emma a question"

agent = 'agent2'
name = state_dict[agent]['name']

# Update the master chat history
# Sender: User, Emma or Liam
update_master_chat_history(sender, message)

# Format the content
content = {"chat_history": state_dict["master_chat_history"], "message": message}
content = str(content)

# Add the message to the agent's chat history - OpenAi format
input_message = {"role": "user", "content": content}
state_dict[agent]["agent_chat_history"].append(input_message)

# Prompt the chat_agent
response = run_chat_agent(state_dict[agent]["agent_chat_history"])

# Set the last_message in the state_dict
state_dict["last_message"] = response

# Update the agent's chat history
input_message = {"role": "assistant", "content": response}
state_dict[agent]["agent_chat_history"].append(input_message)

# Update the master chat history
update_master_chat_history(name, response)


---CHAT AGENT---
Num tokens in agent prompt: 286
Num messages in agent history: 2
(Adjusting my spectacles and leaning slightly towards Emma with a playful smile)

Well, Emma, this whole phenomenon of virtual girlfriends is fascinating, isn't it? It’s certainly sparking a lot of conversation. But I can't help but wonder, from a psychological perspective, how much of this echoes historical patterns of idealized companionship? We've seen throughout history, from courtly love traditions to the rise of romantic novels, people crafting and pursuing *idealized* versions of relationships. Do you see parallels in how people construct and seek these virtual companions, compared to, say, the elaborate fantasies of medieval knights pursuing unattainable ladies, or the carefully curated romantic heroes of Victorian literature? It's almost as if we're all just updating the playbook for longing, isn't it?


In [18]:
#state_dict["last_message"]

In [19]:
route_to = run_router_agent(router_system_message)

---ROUTER AGENT---
Num tokens in router prompt: 712
Num messages in master chat history: 2
Route to...
Name: Emma
agent_id: agent1


In [20]:
# Emma responds to Liam's question
# ---------------------------------

sender = "Liam"
message = response

# Message directed to...
agent = 'agent1' # Emma
name = state_dict[agent]['name']

# Update the master chat history
update_master_chat_history(sender, message)

# Format the content
content = {"chat_history": state_dict["master_chat_history"], "message": state_dict["last_message"]}
content = str(content)

# Add the message to the agent's chat history - OpenAi format
input_message = {"role": "user", "content": content}
state_dict[agent]["agent_chat_history"].append(input_message)

# Prompt the chat_agent
response = run_chat_agent(state_dict[agent]["agent_chat_history"])

# Set the last_message in the state_dict
state_dict["last_message"] = response

# Update the agent's chat history
input_message = {"role": "assistant", "content": response}
state_dict[agent]["agent_chat_history"].append(input_message)

# Update the master chat history
update_master_chat_history(name, response)

---CHAT AGENT---
Num tokens in agent prompt: 828
Num messages in agent history: 2
(Smiling warmly and nodding thoughtfully) That's a wonderfully insightful question, Liam. You’re absolutely right to draw those parallels to historical patterns of idealized companionship. It’s a very astute observation.

From a psychological standpoint, what we’re seeing with virtual girlfriends isn’t entirely new. Humans have always sought connection and fulfillment, and sometimes, that manifests as a longing for an idealized version of a partner. The medieval knights and Victorian heroes you mentioned represent a similar yearning – a desire for someone who embodies perfection, who fulfills a specific set of expectations. 

What's perhaps unique about virtual girlfriends is the level of control and customization available. It allows individuals to create a companion who perfectly aligns with their desires and avoids the complexities and potential disappointments of real-world relationships. 

However, i

In [21]:
#state_dict["last_message"]

In [22]:
route_to = run_router_agent(router_system_message)

---ROUTER AGENT---
Num tokens in router prompt: 1235
Num messages in master chat history: 4
Route to...
Name: User


In [23]:
#state_dict['agent1']

# Run a chat loop

In [24]:
# Initialize the state
#state_dict = initialize_the_state()
print()
print("=== AI GROUP CHAT ===")
print()

print("The scene is a panel discussion.")
print("You are the discussion moderator.")
print(f"The other members of the panel are {AGENT1_NAME}, a psychologist, and {AGENT2_NAME}, an historian.")
print()

print('What topic would you like to discuss?')
print('Examples:')
print("- The rise of virtual girlfriends")
print("- Are we living in a simulation?")
print("- Lessons from the Roman Empire")
print()
discussion_topic = input("Topic: ")

# Initialize the state
state_dict = initialize_the_state(discussion_topic)

j = 0

while True:

    if j == 0:

        print()
        print("Chat launched successfully.")
        print("Please say something to the other members of the panel...")
        print()

        print('---USER---')

        user_input = input("User: ")

        if user_input.lower() == 'q':
            print("Exiting the loop. Goodbye!")
            break  # Exit the loop

        message = user_input

        # Set the name of the speaker
        state_dict['last_speaker'] = "User"

        # Update the master chat history
        update_master_chat_history('user', message)

        # Set the last_message in the state_dict
        # This is the last message that was spoken in this discussion.
        state_dict["last_message"] = message

        # Choose the next speaker
        route_to = run_router_agent(router_system_message)

        j = 1

    else: 

        # Choose the next speaker
        route_to = run_router_agent(router_system_message)

    
    if route_to == "agent1": # Emma

        # Message directed to...
        agent = 'agent1' # Emma
        name = state_dict[agent]['name']

        # Set the name of the speaker
        state_dict['last_speaker'] = name

        # Only use the last 5 messages in the master chat history.
        # This is aimed at keeping the router from failing because
        # the context is too large.
        master_chat_history = state_dict["master_chat_history"]
        master_chat_history =  master_chat_history[-MAX_LEN_MASTER_CHAT_HISTORY:]
        
        # Format the content
        content = {"chat_history":  master_chat_history, "message": state_dict["last_message"]}
        content = str(content)
        
        # Add the message to the agent's chat history - OpenAi format
        input_message = {"role": "user", "content": content}
        state_dict[agent]["agent_chat_history"].append(input_message)

        # Get the num tokens in the prompt
        num_tokens = get_num_tokens(str(state_dict[agent]["agent_chat_history"]))
        print(f'num_tokens_in_context: {num_tokens}')
        
        # Prompt the chat_agent
        response = run_chat_agent(state_dict[agent]["agent_chat_history"])

        # Set the last_message in the state_dict
        state_dict["last_message"] = response
        
        # Update the agent's chat history
        input_message = {"role": "assistant", "name": name, "content": response}
        state_dict[agent]["agent_chat_history"].append(input_message)
        
        # Update the master chat history
        update_master_chat_history(name, response)

        
    elif route_to == "agent2": # Liam
        
        # Message directed to...
        agent = 'agent2' # Liam
        name = state_dict[agent]['name']

        # Set the name of the speaker
        state_dict['last_speaker'] = name

        # Only use the last 5 messages in the master chat history.
        # This is aimed at keeping the router from failing because
        # the context is too large.
        master_chat_history = state_dict["master_chat_history"]
        master_chat_history =  master_chat_history[-MAX_LEN_MASTER_CHAT_HISTORY:]
        
        # Format the content
        content = {"chat_history": master_chat_history, "message": state_dict["last_message"]}
        content = str(content)
        
        # Add the message to the agent's chat history - OpenAi format
        input_message = {"role": "user", "content": content}
        state_dict[agent]["agent_chat_history"].append(input_message)

        # Get the num tokens in the prompt
        num_tokens = get_num_tokens(str(state_dict[agent]["agent_chat_history"]))
        print(f'num_tokens_in_context: {num_tokens}')
        
        # Prompt the chat_agent
        response = run_chat_agent(state_dict[agent]["agent_chat_history"])

        # Set the last_message in the state_dict
        state_dict["last_message"] = response
        
        # Update the agent's chat history
        input_message = {"role": "assistant", "name": name, "content": response}
        state_dict[agent]["agent_chat_history"].append(input_message)
        
        # Update the master chat history
        update_master_chat_history(name, response)

        
    else:

        print('---USER---')
        
        user_input = input("User: ")

        if user_input.lower() == 'q':
            print("Exiting the loop. Goodbye!")
            break  # Exit the loop

        # Set the name of the speaker
        state_dict['last_speaker'] = "User"

        # Update the master chat history
        update_master_chat_history('User', user_input)

        # Set the last_message in the state_dict
        state_dict["last_message"] = user_input




=== AI GROUP CHAT ===

The scene is a panel discussion.
You are the discussion moderator.
The other members of the panel are Emma, a psychologist, and Liam, an historian.

What topic would you like to discuss?
Examples:
- The rise of virtual girlfriends
- Are we living in a simulation?
- Lessons from the Roman Empire



Topic:  Are we living in a simulation?



Chat launched successfully.
Please say something to the other members of the panel...

---USER---


User:  Hi everyone welcome to our discussion. Its a pleasure to have you here


---ROUTER AGENT---
Num tokens in router prompt: 530
Num messages in master chat history: 1
Route to...
Name: Emma
agent_id: agent1
num_tokens_in_context: 313
---CHAT AGENT---
Num tokens in agent prompt: 313
Num messages in agent history: 2
Hi! It's lovely to be here too. It's a fascinating topic to explore, and I'm really looking forward to hearing everyone's perspectives. It takes a lot of courage to even consider such a big question, and I appreciate the openness and willingness to engage with it. I'm here to offer a bit of a different lens, focusing on how these kinds of thoughts and questions can impact our mental and emotional well-being.
---ROUTER AGENT---
Num tokens in router prompt: 646
Num messages in master chat history: 2
Route to...
Name: Liam
agent_id: agent2
num_tokens_in_context: 491
---CHAT AGENT---
Num tokens in agent prompt: 491
Num messages in agent history: 2
(Adjusts spectacles with a flourish)

Well, hello there! It's a genuine pleasure to be here. A simulation, y

...

---ROUTER AGENT---
Num tokens in router prompt: 1504
Num messages in master chat history: 5
Route to...
Name: Liam
agent_id: agent2
num_tokens_in_context: 8248
---CHAT AGENT---
Num tokens in agent prompt: 4963
Num messages in agent history: 7
(Lets out a soft chuckle, adjusts his spectacles, leans back, a thoughtful expression on his face)

Ah, the Occam’s Razor argument! The simplest explanation is often the correct one. You’re absolutely right to bring that up. It’s a wonderfully grounding perspective in the face of such dizzying speculation.

(Strokes his chin)

History is full of examples of elaborate conspiracies that ultimately turned out to be… well, misunderstandings, exaggerations, or simply the product of overactive imaginations. The Tulip Mania in the 17th century, for instance. People genuinely believed that tulip bulbs were incredibly valuable, driving prices to absurd levels. It was a collective delusion, fueled by greed and a lack of critical thinking.

(Pauses, a wry sm

User:  Thank you. Can we have some closing words from you both please


---ROUTER AGENT---
Num tokens in router prompt: 1490
Num messages in master chat history: 5
Route to...
Name: Emma
agent_id: agent1
num_tokens_in_context: 3586
---CHAT AGENT---
Num tokens in agent prompt: 3517
Num messages in agent history: 7
Absolutely. It's been a truly fascinating discussion!

From my perspective as a psychologist, I’m reminded that exploring these big, abstract questions, whether it's the possibility of a simulation or the nature of reality itself, can be incredibly valuable for our mental well-being. It encourages us to think critically, to question our assumptions, and to connect with our sense of wonder. However, it's also crucial to be mindful of how these thoughts might be impacting us emotionally. If they become overwhelming or distressing, please reach out for support. There's strength in vulnerability, and seeking help is a sign of courage, not weakness. Remember to practice self-compassion and to focus on what you *can* control – your thoughts, your action

User:  q


Exiting the loop. Goodbye!


In [25]:
#state_dict["last_message"]

In [26]:
#state_dict["agent2"]

In [27]:
#len(state_dict["agent1"]['agent_chat_history'])